In [5]:
import portfolioTheory as pt
import yftickers as ticks
import datetime as dt

In [6]:
tickers = ticks.ftse100
start=dt.datetime(2000,1,1)
end=dt.datetime(2023,1,1)
threshold=0.9

In [20]:
portfolio = pt.Portfolio(tickers, start, end)
portfolio.getData()
portfolio.cleanData(threshold=threshold)
portfolio.calculate_stats()

[*********************100%***********************]  101 of 101 completed

3 Failed downloads:
- MGGT.L: No timezone found, symbol may be delisted
- RMG.L: No timezone found, symbol may be delisted
- AUTO: No timezone found, symbol may be delisted
33 columns were removed because there were less observations than the threshold (0.9):
AAL       73.20%
ABDN.L    70.15%
AHT       82.06%
AUTO       0.00%
BME       75.34%
BRBY.L    87.60%
CCH.L     41.20%
CPG       81.42%
EDV       63.31%
ENT.L     76.87%
EXPN.L    69.06%
FLTR      49.55%
FRES.L    62.34%
GLEN.L    49.39%
HIK.L     73.07%
HL.L      66.55%
IAG       82.57%
IHG       83.66%
III       67.39%
INVR.L    61.61%
ITRK.L    88.18%
LAND      42.11%
MGGT.L     0.00%
MNDI.L    66.01%
NWG       64.48%
OCDO.L    52.96%
PHNX.L    55.82%
PRU       89.25%
RMG.L      0.00%
RMV.L     71.53%
SKG.L     67.24%
SMIN      46.18%
UTG       79.96%
dtype: object


In [21]:
portfolio.logReturns

,ABF.L,ADM,ANTO.L,AON,AV.L,AVV.L,AZN,BA.L,BARC.L,BATS.L,...,STAN.L,STJ.L,SVT,TSCO,TW.L,ULVR.L,UU.L,VOD,WPP,WTB.L
Date,,,,,,,,,,,,,,,,,,,,,
2001-07-23,0.002736,-0.044287,0.000000,0.000864,-0.002039,0.000000,-0.042977,0.019365,0.013787,-0.001818,...,0.002436,-0.003591,0.000000,0.000484,-0.008075,0.006745,-0.002293,0.050762,-0.003869,-0.024528
2001-07-24,-0.016529,-0.012149,-0.003454,0.007461,-0.009741,-0.010464,-0.015645,-0.030601,-0.009011,0.000909,...,-0.007326,0.013103,0.000000,0.063327,-0.032970,-0.006745,-0.006912,-0.044211,-0.015425,-0.006037
2001-07-25,0.017622,0.006852,-0.013937,0.000571,0.004626,0.000000,-0.017792,0.008439,-0.041835,-0.014652,...,-0.024815,-0.032474,0.000000,0.006795,-0.048651,0.000000,-0.006184,-0.008069,0.021528,-0.003792
2001-07-26,0.004357,-0.009912,-0.003515,0.000000,-0.002054,-0.002110,0.046424,0.009062,-0.031797,0.030884,...,-0.010101,-0.012300,0.000000,0.004504,0.002928,-0.006791,-0.007785,-0.005076,0.029382,-0.000760
2001-07-27,0.019377,-0.003839,0.000000,-0.002575,0.017321,-0.007430,-0.002018,0.009806,0.016782,-0.000895,...,0.031233,0.007398,0.000000,-0.015855,0.008734,-0.005980,0.006233,0.047214,-0.011689,0.001520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-21,0.021161,0.014209,0.026642,0.017376,0.019100,0.001557,0.002654,0.010750,0.014913,0.011698,...,0.007378,0.032549,-0.023311,0.019084,0.030936,0.016346,0.007890,0.005923,0.022619,0.017068
2022-12-22,-0.004770,-0.008308,-0.014858,-0.009549,-0.007372,-0.000934,0.002648,0.001394,-0.005964,0.006180,...,-0.001279,-0.019586,0.000000,-0.005617,-0.004389,-0.002844,-0.012674,-0.007905,-0.007686,-0.010206
2022-12-23,0.011726,0.012014,0.004869,0.001465,-0.004494,0.003110,-0.001470,-0.007690,0.003811,-0.000904,...,0.000000,0.003673,0.064833,0.018647,0.008275,0.000237,-0.003440,0.003960,0.007686,-0.003044
